# MODEL TESTING

The purpose of this notebook is to test different model learning configurations. Runs are saved in the standard _runs_ directory and can be referenced by their unique ID (directory name). Each run-directory contains the following files from training:
- manager.pkl: run manager object containing all settings of the model run - see implementation _/helpers/run_manager.py_ for details
- checkpoint_last.pth: dict containing the most recent state dict for both the model and optimizer (keys: epoch, model, optimizer)
- checkpoint_best.pth: same dict as above, but the best performing model over the entire run based on loss
- ... and randomly named files from the tensorboard summary writer

In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import os
import shutil
import numpy as np
import pandas as pd

import models
import datasets
from helpers import io, run_manager, tester

In [12]:
EXP_ID = "trees_points_full_final"

DATA_PATH = "/home/jovyan/work/processed/256x256"
SAVE_PATH = "/home/jovyan/work/runs"

EM = io.loadPKL(os.path.join(SAVE_PATH, EXP_ID, "manager.pkl"))
TB = SummaryWriter(EM.save_path)

## Load Model from Run Config

In [20]:
model = models.getNet(EM.net_name, EM.n_classes).cuda()

checkpoint = torch.load(os.path.join(EM.save_path, 'checkpoint_best.pth'))
model.load_state_dict(checkpoint['model'])

print("Model loaded...")

Model loaded...


## Load Testing Data

In [17]:
# load image-lists from files
images_path = os.path.join(DATA_PATH, 'image_sets_'+EM.object_type, 'shapes.txt')
images_list = [name.replace("\n","") for name in io.readText(images_path)]
test_size = round(len(images_list) * 0.5)
test_images = images_list[test_size:]

print(f"Dataset sizes: \n - test: {len(test_images)}")

Dataset sizes: 
 - test: 1108


In [18]:
test_set = datasets.getDataset(name = 'denmark_shapes',
                               path = DATA_PATH,
                               images = test_images,
                               object_type = EM.object_type,
                               n_classes = EM.n_classes,
                               transform = EM.transform)

test_sampler = torch.utils.data.SequentialSampler(test_set)

test_loader = DataLoader(test_set, sampler = test_sampler,
                         batch_size = 16,
                         num_workers = 2)

print("Dataloaders ready...")

Dataloaders ready...


## Run Tests

Currently supported metrics are:
- mean Intersection-over-Union (_mIoU_)
- Dice-Score (_dice_)
- Pixel Accuracy (_pAccuracy_)

In [21]:
metric_list = ["mIoU", "dice", "pAccuracy"] #mIoU, dice, pAccuracy

results_dict = tester.testModel(model, test_loader, metric_list)
tester.imagesToTB(model, test_loader, TB, 1)

EM.addTest(results_dict)
io.savePKL(os.path.join(EM.save_path, 'manager.pkl'), EM)

TB.close()
print(pd.DataFrame(results_dict, index=[0]))

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

       mIoU      dice  pAccuracy
0  0.103486  0.182219   0.958697


# EXPLORATION REGION